# Ipython Canvas Experiment

Break away from using tkinter canvas and see if the same functionality can be created using only Ipython widgets.

## Research

### ipycanvas
https://github.com/martinRenou/ipycanvas
This tool came up earlier in research phase. Unused because the canvas drawing style was discontinuous and dot-like, making it look less like smooth handwriting.

### list of widgets
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

### alternatives to tkinter canvas
https://wiki.tcl-lang.org/page/Alternative+Canvases


# Ipycanvas (Revisitied)

Originally found in **experiments/Testing ipycanvas**. We can recreate the canvas code here and see if there are changes we can adapt to make it as deployable as the tkinter prototype.

**How does this compare to the tkinter code?**

- **tkinter:** 
    - fast
    - is not embedded: b/c it uses its own way of poppinp up a window
    - binds command to button 
    - instantiates tkinter canvas

- **ipycanvas:** 
    - slow
        - tends to be marginally slower than the tkinter canvas
    - is embedded
    - roughcanvas 
    - uses roughcanvas.RoughCanvas 
    - not sure if it can support other functionality as tkinter's canvas such as button saving 

**Development**

This tool is working out okay, despite any cons. It is capable of embedding a can as immediately in a cell.  
Functionality is being added to send this canvas for prediction using previously built code.

In [29]:
import os
import base64

import ipywidgets as widgets
from ipywidgets import Image
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox, VBox

from ipycanvas import RoughCanvas, hold_canvas

import PIL
from PIL import Image, ImageDraw

import requests
import json

import configs.configs as cfg

In [30]:
# Mathpix API key details.
app_id = cfg.math_pix_key["app_id"]  
app_key = cfg.math_pix_key["app_key"]

In [31]:
def ocr_request(filename):
    """Sends an API request to MathPix API to be handled by the OCR .
    
    :param filename: Filename of the image to be sent to the MathPix API for a LaTeX prediction.
    :type filename: str
 
    :return: Latex-formatted string.
    :rtype: str
    """
    
    dict_request={
            "src": "data:image/png",
            "formats": ["text", "data", "html"],
            "data_options": {
            "include_asciimath": True,
            "include_latex": True
            }
        }

    # Put desired filename from earlier.
    image_uri = "data:image/png;base64," + base64.b64encode(open(filename, "rb").read()).decode()

    # Send the request.
    r = requests.post("https://api.mathpix.com/v3/text",
                      data=json.dumps({'src': image_uri}),
                      headers={"app_id": app_id, 
                               "app_key": app_key,
                               "Content-type": "application/json"})
    
    json_return = json.loads(r.text)
    latex_return = json_return.get("latex_styled")
    
    return latex_return

In [39]:
def btn_save_eventhandler(self):
    filename = "tempfile.jpg"
    canvas_image.save(filename)

def btn_explore_eventhandler(self):
    print("explore - - !")
    init_browse_dir = os.getcwd()
    
    filename["name"] = filedialog.askopenfilename(
        initialdir = init_browse_dir, title = "Select a File",
        filetypes = (("png files",".png"), ("jpg files",".jpg"), ("all files","*.*")))
    
def btn_pred_eventhandler(self, filename="tempfile.jpg"):
    latex_return = ocr_request(filename)
    latex_pred.append(latex_return
                     )
    print("latex_return: ")
    print(latex_return)
    
def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]
    
def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    x_0 = position[0]
    y_0 = position[1]
        
    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(x_0, y_0, x, y)
        position = (x, y)
    shape.append(position)

def on_mouse_up(x, y, canvas_fill=False):
    global drawing
    global position
    global shape
    
    x_0 = position[0]
    y_0 = position[1]
    
    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(x_0-1, y_0-1, x+1, y+1)
        
    for i in range(len(shape)):
        x_0 = shape[i][0]
        y_0 = shape[i][1]
        x_1 = shape[i+1][0]
        y_1 = shape[i+1][1]
        draw.line([x_0, y_0, x_1, y_1],fill="black",width=2)        
    
drawing = False
position = None
shape = []
width = 600
height = 400

latex_pred = []

# Instantiate ipycanvas RoughCanvas and use 
# PIL to create an empty image and draw object to draw on
# memory only, not visible to the user.
canvas = RoughCanvas(width=width, height=height)
canvas_image = PIL.Image.new("L", (width, height), color="white")
draw = ImageDraw.Draw(canvas_image)

# Canvas Actions.
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
canvas.stroke_style = "black"
canvas.line_width = 1.5

# Buttons.
btn_explore = widgets.Button(description="Browse Files")
btn_save = widgets.Button(description="Save Canvas")
btn_pred = widgets.Button(description="Predict LaTeX!")

btn_save.on_click(btn_save_eventhandler)
btn_pred.on_click(btn_pred_eventhandler)
btn_explore.on_click(btn_explore_eventhandler)

display(btn_explore)
display(btn_save)
display(btn_pred)


VBox((canvas, ))

Button(description='Browse Files', style=ButtonStyle())

Button(description='Save Canvas', style=ButtonStyle())

Button(description='Predict LaTeX!', style=ButtonStyle())

### Post production work